In [4]:
%pip install ultralytics
%pip install git+https://github.com/openai/CLIP.git

KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO
import clip
import torch
yolov8 = YOLO("yolov8m-oiv7.pt")
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
from PIL import Image
sample_img = Image.open("hand_pen.jpg")
clip_img = preprocess(sample_img).unsqueeze(0).to(device)
question = "What is in the hand?"
candidates = ["a pen", "an apple", "a handgun", "a phone", "a computer", "a cattle", "a cake"]
input_texts = [f"{question} {candidate}" for candidate in candidates]

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(clip_img)
    qa_features = model.encode_text(clip.tokenize(input_texts).to(device))
    similarity = (100.0 * image_features @ qa_features.T).softmax(dim=-1)

In [ ]:
best_index = similarity.squeeze().argmax().item()
best_answer = candidates[best_index]

print(f"{question}\n{best_answer}")